In [ ]:
import papermill as pm
import pandas as pd
import numpy as np
from multiprocessing import Pool
import concurrent.futures
import queue
import os
from pathlib import Path
from datetime import datetime

In [ ]:
def run_notebook(parameters,notebook_to_run,parameters_common={}):
    
    # change to directory where the notebook is (resolve relative imports)
    os.chdir(Path(notebook_to_run).absolute().parent)
    
    # run notebook
    for parameters_spec in parameters_list:
        parameters = {**parameters_common, **parameters_spec}

        pm.execute_notebook(
            notebook_to_run,
            '/dev/null',
            parameters=parameters)

# 0) Create projections
**In:** folder with 2-channel `.msr` images

**Out:** folder `projections` with maximum inensity projections `.png` images

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/sted_spot_detection/example/"}
]

notebook_to_run = "/home/stumberger/ep2024/subscripts/msr_make_projections_2_color.ipynb"

run_notebook(parameters_list,notebook_to_run)

# 1) Split channels and resave `msr` as `tif`
**In:** folder containing folder `raw` with `.msr` images

**Out:** folder `tif` with `.tif` images split by channel

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/sted_spot_detection/example/"}
]

notebook_to_run = "/home/stumberger/image-analyis-recipes/resave/resave_msr_as_tiff.ipynb"

run_notebook(parameters_list,notebook_to_run)

# 2) Spot detection
**In:** 
- `tif_subfolder` containing single channel `tif` images
- `out_subfolder` containing spot detection settings file `chX.txt` generated via RS-FISH Fiji plugin for each channel specified in `channels`
- `acquisition_info.json` file with the sample metadata which you wish to be added to your spots

**Out:**
- folder containing a `csv` file for each provioded `tif`
- `merge.csv` file combining spots and their metadata from all images in the batch
- vis folder with detection visualization for all `tif`s as `png` (good for checking detection results)

In [ ]:
parameters_list = [
   {"in_path": "/home/stumberger/ep2024/sted_spot_detection/example/"}
]

parameters_common = {"channels": [0,1],
                     "tif_subfolder": "tif",
                    "out_subfolder": "detections"}

notebook_to_run = "/home/stumberger/ep2024/subscripts/RS-FISH_spot_detection.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 3) Get spot pairs and distance distributions
**In:** `.csv` file with spots in 2 channels

**Out:** 
- `merge_distances.csv` file in `detections` folder, containing spot pairs between 2 channels and distances between each pair
- folder `detections_good` with max intensity projections and detections of images with `n_enh` in second channel

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/sted_spot_detection/example/"}
]

parameters_common = {"pixel_size": [0.045, 0.045, 0.06],
                     "limit": 1.5, #max P-E distance to look at [um]
                     "n_enh": 3, 
                     "channels": [0,1],
                     "spot_subfolder": "detections/merge.csv",
                     "projections_subfolder": "detections_good"}

notebook_to_run = "/home/stumberger/ep2024/subscripts/paired_spot_distances_2_channels_sted.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 4) Join csvs (optional)
**In:** Paths to multiples folders containing the `merge_distances.csv` files you want to join

**Out:** `multi_sted_distances_{time}.csv` file with all `.csv`s joined

In [6]:
wd = "/home/stumberger/ep2024/sted_spot_detection/example/"

# directory where csv files you want to merge are located
csv_files = [
    "/home/stumberger/ep2024/sted_spot_detection/example/detections/"
]

# Initialize an empty list to store DataFrames
dataframes = []

# Read and store each CSV file as a DataFrame
for file in csv_files:
    df = pd.read_csv(f"{file}/merge_distances.csv")
    dataframes.append(df)

# Join the DataFrames using Pandas (e.g., concatenate them vertically)
joined_dataframe = pd.concat(dataframes, ignore_index=True)
# Save the joined DataFrame to a new CSV file
time =  datetime.now().strftime("%Y-%m-%d-%H-%M")
joined_dataframe.to_csv(f'{wd}/multi_sted_distances_{time}.csv', index=False)